In [1]:
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON

# Data viz packages
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import isodate
# NLP
import nltk
import ssl


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from wordcloud import WordCloud

#from lib import helpers

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roshnisaravanan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/roshnisaravanan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
api_key = ''

In [3]:
channel_ids = ['UCtYLUTtgS3k1Fg4y5tAhLbw', # Statquest
               'UCCezIgC97PvUuR4_gbFUs5g', # Corey Schafer
               'UCfzlCWGWYyIQ0aLC5w48gBQ', # Sentdex
               'UCNU_lfiiWBdtULKOw6X0Dig', # Krish Naik
               'UCzL_0nIe8B4-7ShhVPfJkgw', # DatascienceDoJo
               'UCLLw7jmFsvfIVaUFsLs8mlQ', # Luke Barousse 
               'UCiT9RITQ9PW6BhXK0y2jaeg', # Ken Jee
               'UC7cs8q-gJRlGwj4A8OmCmXg', # Alex the analyst
               'UC2UXDak6o7rBm23k3Vv5dww', # Tina Huang
              ]

In [4]:
api_service_name = "youtube"
api_version = "v3"


# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
)
response = request.execute()

JSON(response)

<IPython.core.display.JSON object>

In [5]:
def get_channel_stats(youtube, channel_ids):
    """
    Get channel statistics: title, subscriber count, view count, video count, upload playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    channels_ids: list of channel IDs
    
    Returns:
    Dataframe containing the channel statistics for all channels in the provided list: title, subscriber count, view count, video count, upload playlist
    
    """
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data)

In [6]:
channel_data = get_channel_stats(youtube, channel_ids)

In [7]:
channel_data

,channelName,subscribers,views,totalVideos,playlistId
0,Tina Huang,397000,15687499,107,UU2UXDak6o7rBm23k3Vv5dww
1,Corey Schafer,991000,77619402,230,UUCezIgC97PvUuR4_gbFUs5g
2,sentdex,1160000,104710825,1255,UUfzlCWGWYyIQ0aLC5w48gBQ
3,Krish Naik,656000,61848715,1498,UUNU_lfiiWBdtULKOw6X0Dig
4,Data Science Dojo,88100,4994626,360,UUzL_0nIe8B4-7ShhVPfJkgw
5,Ken Jee,218000,6926722,238,UUiT9RITQ9PW6BhXK0y2jaeg
6,Alex The Analyst,284000,11298116,164,UU7cs8q-gJRlGwj4A8OmCmXg
7,StatQuest with Josh Starmer,798000,40937954,221,UUtYLUTtgS3k1Fg4y5tAhLbw
8,Luke Barousse,241000,10934191,79,UULLw7jmFsvfIVaUFsLs8mlQ


In [8]:
playlist_id = 'UULLw7jmFsvfIVaUFsLs8mlQ'

def get_video_ids(youtube, playlist_id):
    """
    Get list of video IDs of all videos in the given playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    playlist_id: playlist ID of the channel
    
    Returns:
    List of video IDs of all videos in the playlist
    
    """
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [9]:
video_ids = get_video_ids(youtube, playlist_id)

In [10]:
len(video_ids)

79

In [11]:
request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId='UU22go5LdQEw-iDuxFb4C0hw'
)
response = request.execute()

JSON(response)

<IPython.core.display.JSON object>

In [12]:
def get_video_details(youtube, video_ids):
    """
    Get video statistics of all videos with given IDs
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with statistics of videos, i.e.:
        'channelTitle', 'title', 'description', 'tags', 'publishedAt'
        'viewCount', 'likeCount', 'favoriteCount', 'commentCount'
        'duration', 'definition', 'caption'
    """
        
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            #return pd.DataFrame(all_video_info)

In [13]:
print(video_ids)
video_df = get_video_details(youtube, video_ids)

['0R1LCYpiQsw', 'AopGnnPPczY', 'tBfIh3VQX2o', 'i8PKI_zJfDU', 'lw6sPHH7twk', '5Q0gB7imNOo', 'MSlIfGSI6bo', 'NQSe-SuykJU', 'W5qAY4XidHE', '09JnFEdZe2A', 'GEBzsz8ZSXs', 'NAuuqdzC_rI', 'CC66RXeTn_4', 'aqRxZSrJLEc', 'U4vh2EClJic', 'egtF-C5WIpk', '1sQDJSQBUe4', 'NrVYzpy4Xns', 'y3TxcejHw-4', '1kU_ASADlPY', 'TeZVQ9hsgTM', 'VrdnBxx8BBI', 'W_mC_I0_fx4', 'Zcy-ND_4ydQ', 'VchXKvfCi3s', 'z5mAumgV2rA', 'cIZWgPmpRVc', '78LMtFItAdw', 'ZWqH5emDdOI', 'BEKXRqqgHCw', '3Ao2FV1r5-8', '3TBwY4VjLX8', 'aw14VK9sN2s', 'DNJfUPfSZpY', 'MfUzKeEKtr8', 'DGrszAeMZJI', 'jp-Lv_3a2VI', 'Ush_KFSKNgg', 'fmLPS6FBbac', '_r655ogzK4A', 'YZkcEtWgMGw', 'axgNfM55dw8', 'GmTD-MBea_E', 'X4DBQTl7AfI', 'hWs_q9NBBKM', 'W6N8n99htvw', '18ZYDDIgIiM', 'Vor_OkEnxAA', 'xzR4L_XYW3g', 'aULGH7zts0c', 'GZKDG2pyKhI', 'sW2w10lL1LM', 'vyPm2fOyS7Y', 'YCcC8LQ5RGU', 'AfG4jS13X5c', 'Dt2hHmTkoMg', 'p04t7Ho-DEM', 'MJOYxUU0wRg', 'WslmhWukJdM', 'MADnq-md0yw', 'NnfbVOt9IR0', 'eCN_GRRAoF8', '-3S_fS4yNA8', 'ip_HcMI0EQQ', 'UuwlySU7Hjg', 'qI0uJsLweoM', 'HvAjnpA6

In [14]:
video_df

In [15]:
def get_comments_in_videos(youtube, video_ids):
    """
    Get top level comments as text from all videos with given IDs (only the first 10 comments due to quote limit of Youtube API)
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with video IDs and associated top level comment in text.
    
    """
    all_comments = []
    
    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
        
            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
            
        except: 
            # When error occurs - most likely because comments are disabled on a video
            print('Could not get comments for video ' + video_id)
        
    return pd.DataFrame(all_comments)   

In [16]:
comments = get_comments_in_videos(youtube, video_ids)

In [17]:
comments

,video_id,comments
0,0R1LCYpiQsw,"[Data Nerds, I need your help!! I'm looking fo..."
1,AopGnnPPczY,"[DATA NERDS, I'M BACK!!!!! 🤓 🙌🏼\nThank y'all s..."
2,tBfIh3VQX2o,[What’s the most advanced math you’ve used wit...
3,i8PKI_zJfDU,[2:46 Morale of the story: always Subscribe to...
4,lw6sPHH7twk,[Data Nerds!! Shoutout to ChartMogul for spons...
...,...,...
74,FQiehwXLKV4,[Can you post a video about how to conditional...
75,IdZuTave8QY,"[Thank you Luke! You are a great instructor!, ..."
76,W_5x9mURZe4,"[Hey Luke, just finished the Google Data Analy..."
77,sgS4TauqhJM,"[Great vid! Completing a cert myself now, Than..."


In [18]:
video_df.isnull().any()

AttributeError: 'NoneType' object has no attribute 'isnull'

In [ ]:
video_df.dtypes

In [19]:
# Convert count columns to numeric columns
numeric_cols = ['viewCount', 'likeCount', 'favouriteCount','commentCount']
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors='coerce',axis=1)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Create publish day (in the week) column
video_df['publishedAt'] =  video_df['publishedAt'].apply(lambda x: parser.parse(x)) 
video_df['pushblishDayName'] = video_df['publishedAt'].apply(lambda x: x.strftime("%A")) 

In [ ]:
# convert duration to seconds
video_df['durationSecs'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_df['durationSecs'] = video_df['durationSecs'].astype('timedelta64[s]')

In [ ]:
# Add number of tags
video_df['tagsCount'] = video_df['tags'].apply(lambda x: 0 if x is None else len(x))

In [ ]:
# Comments and likes per 1000 view ratio
video_df['likeRatio'] = video_df['likeCount']/ video_df['viewCount'] * 1000
video_df['commentRatio'] = video_df['commentCount']/ video_df['viewCount'] * 1000

In [ ]:
# Title character length
video_df['titleLength'] = video_df['title'].apply(lambda x: len(x))

In [ ]:
video_df.head()

In [ ]:
# Create a dataframe with video statistics and comments from all channels

video_df = pd.DataFrame()
comments_df = pd.DataFrame()

for c in channel_data['channelName'].unique():
    print("Getting video information from channel: " + c)
    playlist_id = channel_data.loc[channel_data['channelName']== c, 'playlistId'].iloc[0]
    video_ids = get_video_ids(youtube, playlist_id)
    
    # get video data
    video_data = get_video_details(youtube, video_ids)
    # get comment data
    comments_data = get_comments_in_videos(youtube, video_ids)

    # append video data together and comment data toghether
    video_df = video_df.append(video_data, ignore_index=True)
    comments_df = comments_df.append(comments_data, ignore_index=True)

Getting video information from channel: Tina Huang


/var/folders/pk/x12wsw4d1lv9831hq79tsyvh0000gn/T/ipykernel_940/893411185.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)
/var/folders/pk/x12wsw4d1lv9831hq79tsyvh0000gn/T/ipykernel_940/893411185.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comments_df = comments_df.append(comments_data, ignore_index=True)


Getting video information from channel: Corey Schafer
Could not get comments for video ZB7c00QgmdQ


/var/folders/pk/x12wsw4d1lv9831hq79tsyvh0000gn/T/ipykernel_940/893411185.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)
/var/folders/pk/x12wsw4d1lv9831hq79tsyvh0000gn/T/ipykernel_940/893411185.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comments_df = comments_df.append(comments_data, ignore_index=True)


Getting video information from channel: sentdex
Could not get comments for video XfzbZm89z3o


/var/folders/pk/x12wsw4d1lv9831hq79tsyvh0000gn/T/ipykernel_940/893411185.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)
/var/folders/pk/x12wsw4d1lv9831hq79tsyvh0000gn/T/ipykernel_940/893411185.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comments_df = comments_df.append(comments_data, ignore_index=True)


Getting video information from channel: Krish Naik
Could not get comments for video sjq1OhMzSSU
